# A Transformer-based recommendation system


In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
from src.dataset import RatingDataset
from src import utils
from torch.utils.data import DataLoader
from torch import nn
import torch
from tqdm import tqdm
import numpy as np
import os

## (1) Model Config

In [3]:
# inputs

In [4]:
age_group_id_map_dict = utils.open_object("./artifacts/age_group_id_map_dict.pkl")

movie_id_map_dict = utils.open_object("./artifacts/movie_id_map_dict.pkl")

occupation_id_map_dict = utils.open_object("./artifacts/occupation_id_map_dict.pkl")

sex_id_map_dict = utils.open_object("./artifacts/sex_id_map_dict.pkl")

user_id_map_dict = utils.open_object("./artifacts/user_id_map_dict.pkl")
# genres_map_dict = utils.open_object("./artifacts/genres_map_dict.pkl")

In [5]:
rating_min_max_scaler = utils.open_object("./artifacts/rating_min_max_scaler.pkl")

In [6]:
num_user = len(user_id_map_dict)
num_movie = len(movie_id_map_dict)
num_occupation = len(occupation_id_map_dict)
num_age_group = len(age_group_id_map_dict)
# num_genre = len(genres_map_dict)

In [7]:
embed_configs = {}
EMED_DIM=32
sequence_length=4
embed_configs['user']={"embed_dim":EMED_DIM,"num_embed":num_user}
embed_configs['movie']={"embed_dim":EMED_DIM,"num_embed":num_movie}
embed_configs['occupation']={"embed_dim":EMED_DIM,"num_embed":num_occupation}
embed_configs['age_group']={"embed_dim":EMED_DIM,"num_embed":num_age_group}
embed_configs['position'] = {"embed_dim":EMED_DIM,"num_embed":sequence_length}

In [8]:
config_dict={}
config_dict['embed_configs'] = embed_configs
config_dict['transformer_num_layer']=3
config_dict['dropout']=0.2
config_dict['epoches']=6
config_dict['learning_rate']=0.001
config_dict['batch_size']=64

In [9]:
class Config:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            setattr(self, key, value)

In [10]:
config = Config(dictionary=config_dict)

In [11]:
config.embed_configs

{'user': {'embed_dim': 32, 'num_embed': 6041},
 'movie': {'embed_dim': 32, 'num_embed': 3884},
 'occupation': {'embed_dim': 32, 'num_embed': 22},
 'age_group': {'embed_dim': 32, 'num_embed': 8},
 'position': {'embed_dim': 32, 'num_embed': 4}}

## (3) Load Model




In [12]:
from src.model import BSTRecommender

In [13]:
model = BSTRecommender(config=config)

In [14]:
# loss_func = nn.MSELoss()
loss_func = nn.L1Loss()

## (4) Training

In [15]:
df_train = pd.read_parquet("./artifacts/train_data.parquet")
df_test = pd.read_parquet("./artifacts/test_data.parquet")

In [16]:
train_dataset = RatingDataset(data=df_train) 
test_dataset = RatingDataset(data=df_test)

In [17]:
train_loader = DataLoader(train_dataset,batch_size=config.batch_size,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=config.batch_size,shuffle=True)

In [18]:
from src.evaluation import evaluate

In [19]:
before_training_metrics = evaluate(model,test_loader,loss_func=loss_func)
print(before_training_metrics)

100%|██████████| 1166/1166 [00:05<00:00, 199.16it/s]

{'eval_loss': 0.24997990546463694}


In [20]:
optimizer = torch.optim.AdamW(model.parameters(),lr=config.learning_rate)

In [21]:
total_batch = 0
best_eval_loss =  float("inf")
best_checkpoint = 0

In [22]:
model_version='v1'

In [23]:
config.eval_steps = 2000

In [24]:
# Train
total_pbar = tqdm(total=len(train_loader)*config.epoches,
                  desc="Training", position=0, leave=True)

metrics_list = []

for epoch in range(config.epoches):
    # print("*"*50 + f"epoch: {epoch + 1}" + "*"*50)

    train_loss_list = []
    prob_list = []
    rating_list = []

    for inputs in train_loader:
        model = model.train()
        optimizer.zero_grad()
        probs = model(inputs)

        rating = inputs['target_rating'].view(-1, 1)

        loss = loss_func(probs, rating)
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())

        probs = probs.detach().cpu().numpy().flatten().tolist()
        prob_list.extend(probs)
        rating = rating.numpy().flatten().tolist()
        rating_list.extend(rating)

        if (total_batch+1) % config.eval_steps == 0:

            improve = False
            model_metrics = evaluate(model, test_loader)
            eval_loss = model_metrics['eval_loss']

            if eval_loss <= best_eval_loss:
                improve = True
                best_checkpoint = total_batch+1
                best_eval_loss = eval_loss

            train_loss = np.mean(train_loss_list)
            
            model_metrics['best_eval_loss'] = best_eval_loss
            model_metrics['train_loss'] = train_loss
            model_metrics["steps"] = total_batch+1
            model_metrics["best_checkpoint"] = best_checkpoint
            metrics_list.append(model_metrics)
                
            if improve:
                save_dir = os.path.join("model", model_version)
                os.makedirs(save_dir, exist_ok=True)
                model_path = utils.save_model(model, save_dir, total_batch+1, model_metrics)
            
            post_fix_message = {k:round(v,3) for k,v in model_metrics.items()}
            total_pbar.set_postfix(post_fix_message)


            model = model.train()

        total_batch += 1
        total_pbar.update(1)

    model = model.train()

total_pbar.close()

 81%|████████  | 945/1166 [00:05<00:01, 197.15it/s] 82.32it/s, eval_loss=0.178, best_eval_loss=0.178, train_loss=0.185, steps=6000, best_checkpoint=6000]  